In [8]:
#import dependencies
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import polars as pl


In [4]:
#create a file path and read the CSV
file_path=Path('../raw_data/IMBD.csv')
IMBD_df=pd.read_csv(file_path,encoding='UTF8')
#drop duplicate title entries
IMBD_df=IMBD_df.drop_duplicates(subset='title')
#drop nan values
IMBD_df.dropna(inplace=True)
#take the primary genre of movies listed with multiple genres.
IMBD_df['primary_genre'] = IMBD_df['genre'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else None)

#define bins for votes
bins = [0,20000,40000,60000,80000,100000]
#name each bracket 
group_names = ["0-20000","20000-40000", "40000-60000","60000-80000" ,"80000-100000"]
merged_df_new = IMBD_df.copy()
merged_df_new["votes"] = merged_df_new["votes"].str.replace(",","")
merged_df_new['votes'] = pd.to_numeric(merged_df_new['votes'], errors='coerce')

# Use `pd.cut` to categorize spending based on the bins.
merged_df_new["vote_Range"] = pd.cut(merged_df_new["votes"],bins,labels=group_names)

#rearrange columns and drop nas in the vote_range
merged_df_cleaned = merged_df_new[['title','year','duration','primary_genre','rating','vote_Range']]
merged_df_cleaned = merged_df_cleaned.dropna(subset=['vote_Range'])

#write to CSV
merged_df_cleaned.to_csv('../clean_data_sets/IMBD_Cleaned.csv', index=False)

In [7]:
#create filepath and read CSV
file_path_2=Path('../raw_data/MoviesOnStreamingPlatforms_updated.csv')
mosp_df=pd.read_csv(file_path_2,encoding="UTF8")
#change name of title column
mosp_df=mosp_df.rename(columns={"Title":'title'})
mosp_df=mosp_df.drop_duplicates(subset='title')
mosp_df.dropna(inplace=True)
#define bins and label them in groups
bins = [0,30,60,90,120,180,240]
group_names = ['0-30','30-60','60-90','90-120','120-180','180-240']
df_runtime = mosp_df.copy()
#make runtime a numeric value for further analysis
df_runtime['Runtime'] = pd.to_numeric(df_runtime['Runtime'], errors='coerce')

#take the primary genre of a movie, in lieu of listing it with multiple genres
df_runtime['primary_genre'] = df_runtime['Genres'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else None)
# Use `pd.cut` to categorize spending based on the bins.
df_runtime["Runtime_Range"] = pd.cut(df_runtime['Runtime'],bins,labels=group_names)
#rearrange columns
merged_df_clean = df_runtime[['ID','title','Year','Age','IMDb','primary_genre','Netflix','Hulu','Prime Video','Disney+','Type','Runtime_Range']] 
#further cleaning and creating ID column for compatibility in SQL
merged_df_clean = merged_df_clean.dropna(subset=['Runtime_Range'])
merged_df_clean = merged_df_clean[merged_df_clean['title'].isin(merged_df_cleaned['title'])]
merged_df_clean['ID'] = range(1, len(merged_df_clean) + 1)
#write to a CSV
merged_df_clean.to_csv('../clean_data_sets/Movies_Cleaned.csv', index=False)


In [10]:
#use polars to extract data from CSV
polars_df=pl.read_csv('../clean_data_sets/Concise_Data.csv')
#transform data into Dataframe
new_runtime_df=polars_df[['title','rating','Runtime']]
#write the polars dataframe to a CSV.
new_runtime_df.write_csv("../clean_data_sets/Runtime_Data.csv")